# Import libraries

In [ ]:
import warnings
warnings.filterwarnings( 'ignore' )

In [ ]:
# Built-in libraries
#
import pandas as pd
import os
from   tqdm    import tqdm
from   os      import listdir
from   os.path import isdir, isfile, join

import librosa
import librosa.display

# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
#
# Visualization Libraries
#
import matplotlib 
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn           as sns

# User libraries
#
from utils.utils         import *

In [ ]:
sr = 192000
audio_data  = librosa.load('./Data/S1Ep3_Sound/Dev_2/IMP23ABSU_MIC.wav', sr = sr)

In [ ]:
plt.figure(figsize=(25, 7))

librosa.display.waveplot( audio_data[0],  audio_data[1],  alpha=0.7 )

# Spectrogram
 
A spectrogram is a visual way of representing the signal strength, or “loudness”, of a signal over time at various frequencies present in a particular waveform. Not only can one see whether there is more or less energy at, for example, 2 Hz vs 10 Hz, but one can also see how energy levels vary over time.

In [ ]:
X   = librosa.stft( audio_data[0] )
Xdb = librosa.amplitude_to_db(abs(X))


plt.figure(figsize=(25, 7))
librosa.display.specshow(Xdb, sr=audio_data[1], x_axis='time', y_axis='hz')
plt.colorbar()

# Mel-Frequency Cepstral Coefficients(MFCCs)

The Mel frequency cepstral coefficients (MFCCs) of a signal are a small set of features (usually about 10–20) which concisely describe the overall shape of a spectral envelope. It models the characteristics of the human voice.

In [ ]:
mfccs = librosa.feature.mfcc(audio_data[0], sr=audio_data[1])

#Displaying  the MFCCs:
#
plt.figure(figsize=(15, 7))
librosa.display.specshow(mfccs, sr=audio_data[1], x_axis='time')

# Chroma feature

A chroma feature or vector is typically a 12-element feature vector indicating how much energy of each pitch class, {C, C#, D, D#, E, …, B}, is present in the signal. In short, It provides a robust way to describe a similarity measure between music pieces.

In [ ]:
chromagram = librosa.feature.chroma_stft( audio_data[0], sr=audio_data[1] )

plt.figure(figsize=(15, 5))
librosa.display.specshow( chromagram, x_axis='time', y_axis='chroma', cmap='coolwarm')

# Feature extraction from Audio signal

## Spectral Centroid

The spectral centroid indicates at which frequency the energy of a spectrum is centered upon or in other words It indicates where the ” center of mass” for a sound is located. This is like a weighted mean:

In [ ]:
import sklearn
# Normalising the spectral centroid for visualisation
#
def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)





spectral_centroids = librosa.feature.spectral_centroid(audio_data[0][:sr//100], sr=audio_data[1])[0]


# Computing the time variable for visualization
#
plt.figure(figsize=(25, 7))
frames = range(len(spectral_centroids))
t      = librosa.frames_to_time(frames)


#Plotting the Spectral Centroid along the waveform
#
librosa.display.waveplot(audio_data[0], sr=audio_data[1], alpha=0.4)
plt.plot(t, normalize(spectral_centroids), color='b')

In [ ]:
audio_data[0].shape

In [ ]:
spectral_centroids.shape  # OLA -> 44825

## Spectral Rolloff

It is a measure of the shape of the signal. It represents the frequency at which high frequencies decline to 0. To obtain it, we have to calculate the fraction of bins in the power spectrum where 85% of its power is at lower frequencies.

In [ ]:
spectral_rolloff = librosa.feature.spectral_rolloff(audio_data[0]+0.01, sr=audio_data[1])[0]


plt.figure(figsize=(25, 7))
librosa.display.waveplot(audio_data[0], sr=audio_data[1], alpha=0.4)
plt.plot(t, normalize(spectral_rolloff), color='r')

## Spectral Bandwidth

The spectral bandwidth is defined as the width of the band of light at one-half the peak maximum (or full width at half maximum [FWHM]) and is represented by the two vertical red lines and λSB on the wavelength axis.

In [ ]:
spectral_bandwidth_2 = librosa.feature.spectral_bandwidth(audio_data[0]+0.01, sr=audio_data[1], p=2)[0] # default
spectral_bandwidth_3 = librosa.feature.spectral_bandwidth(audio_data[0]+0.01, sr=audio_data[1], p=3)[0]
spectral_bandwidth_4 = librosa.feature.spectral_bandwidth(audio_data[0]+0.01, sr=audio_data[1], p=4)[0]



plt.figure(figsize=(25, 7))
#
librosa.display.waveplot(audio_data[0], sr=audio_data[1], alpha=0.4)
plt.plot(t, normalize(spectral_bandwidth_2), color='r')
plt.plot(t, normalize(spectral_bandwidth_3), color='g')
plt.plot(t, normalize(spectral_bandwidth_4), color='y')
plt.legend(('p = 2', 'p = 3', 'p = 4'))

## Zero-Crossing Rate

A very simple way for measuring the smoothness of a signal is to calculate the number of zero-crossing within a segment of that signal. A voice signal oscillates slowly — for example, a 100 Hz signal will cross zero 100 per second — whereas an unvoiced fricative can have 3000 zero crossings per second.

In [ ]:
zero_crossings = librosa.zero_crossings(audio_data[0][:1000], pad=False)
print(sum(zero_crossings))

# Import data

In [ ]:
# Define path
#
path              = './Data/S1Ep3_Sound/'
SoundData         = {}


for i in range(2, 6 ):
    
    legend = 'Dev_{}'.format(i)
    print('[INFO] Case: ', legend)
    

    # Load Accelerometer data
    #
    df = pd.read_csv( path + 'Dev_{}/IMP23ABSU_MIC.csv'.format(i))
    print('[INFO] Sound data loaded')

    
   
    
    
    # Import specifications
    #
    AcquisitionInfo = pd.read_json(path + 'Dev_{}/AcquisitionInfo.json'.format(i))
    DeviceConfig    = pd.read_json(path + 'Dev_{}/DeviceConfig.json'.format(i))
    
    print('[INFO] Name:   ', DeviceConfig['device']['deviceInfo']['alias'])
    print('[INFO] Serial: ', DeviceConfig['device']['deviceInfo']['serialNumber'])
    
    
    
    if ( (df['Time'].diff()[1:] <= 0.0).any() ):
        print('[ERROR] Data are probably not correct')
    else:
        # Fix date-time and set index 
        #
        df['Time_index']            = pd.to_timedelta( df['Time'], 's')
        
        # Set index
        #
        df.set_index('Time_index', inplace=True)    

        

        
        
        
        
#     # Plot measurements
#     #
#     plt.figure( figsize=(25, 5) )    
#     df[ 'MIC' ].plot( alpha=0.7 )        
    
#     plt.title( legend )
#     plt.show()

    break
   
    
    
    # Store data
    #
    SoundData[ legend ]     = df


In [ ]:
Signal = df['MIC'].to_numpy()

Sample_rate = 52000

Frequencies = np.linspace (0.0, Sample_rate//2, Signal.shape[0]//2)
freq_data   = fft( Signal )
Amplitudes  = np.abs (freq_data [0:Signal.shape[0]//2])



plt.figure( figsize=(25,7) )
plt.plot(Frequencies, Amplitudes)